# Max Flow Applications

The purpose of this assignment is to investigate applications of finding a Max Flow. The problem asks you to design and implement an algorithm for shipping a material between nodes with different supply and demand requirements.

## Movie distribution

First solve Problem 3 from HW3-theoretical. 

Now suppose a movie distributor would like to ship a copy of a film from CA to every other state. There are therefore 48 units to ship out of CA, and each other state receives 1 unit. 

The dataset contiguous-usa.dat lists the adjacent states in the US. Each line lists two adjacent states; thus AK and HI are omitted, but DC is included in the data. The following code reads in the graph of US states.

In [220]:
import networkx as nx
G = nx.Graph()
usa = open('contiguous-usa.dat')
for line in usa:
    s1, s2 = line.strip().split()
    G.add_edge(s1, s2)

We now encode the demands into the graph.

In [221]:
for state in G.nodes():
    if state != 'CA':
        G.node[state]['demand'] = 1
G.node['CA']['demand'] = -48

We will assign a uniform capacity of 16 to each edge. Since CA has only three adjacent states, this is the smallest possible uniform capacity that allows one to ship all 48 units out of CA. As we have created an undirected graph, and flows have directions, we first convert the graph to a directed graph.

In [222]:
G = nx.DiGraph(G)
uniform_capacity = 16
for (s1, s2) in G.edges():
    G.edge[s1][s2]['capacity'] = uniform_capacity

Complete the following function to implement your algorithm to find a flow with demands. Your function should work correctly for any input, not just the movie instance considered here. As always, you are encouraged to define auxiliary functions as needed for clarity.

In [223]:
def preprocess(graph):
    """Plug in source and sink to make the graph suitable for max flow set
    up
    """
    graph.add_node('source')
    graph.add_node('sink')
    graph.node['source']['demand'] = 0
    graph.node['sink']['demand'] = 0
    for key in graph.node:
        if graph.node[key]['demand'] > 0:
            graph.add_edge(key,'sink')
            graph.edge[key]['sink']['capacity'] = graph.node[key]['demand']
        elif graph.node[key]['demand'] < 0:
            graph.add_edge('source',key) 
            graph.edge['source'][key]['capacity'] = - graph.node[key]['demand']
    return graph

In [224]:
def zero_flow(graph):
    flow = {}
    for state in graph.nodes():
        flow[state] = {}
    for (s1,s2) in graph.edges():
        flow[s1][s2] = 0
    return flow

In [225]:
def s_t_path(graph):
    # TODO: implement function
    visited = []
    stack = ['source',]
    parent = {}
    while 'sink' not in stack and stack:
    # pop the component into stack
        state = stack.pop()
        if state not in visited:
    # Append the just visited node to the visited node
            visited.append(state)  
            visitedSet = set(visited)
            children = set(graph.neighbors(state))-set(visited)
    #Keep track of the parent
        for child in children:
            parent[child] = state
    # extend the stack with the new edges discovered
            stack.extend(children)
    flow = []
    if 'sink' not in parent:
        flow.append(0)
        return flow
    state = 'sink'
    cap = graph.edge[parent['sink']]['sink']['capacity']
    while state != 'source':
        flow.insert(0,state)
        if graph.edge[parent['sink']]['sink']['capacity'] < cap:
            cap = graph.edge[parent['sink']]['sink']['capacity']
        state = parent[state]
    flow.insert(0,state)
    flow.append(cap)
    return flow

In [226]:
def add_flow(flow, flow_temp):
    cap = flow_temp[-1]
    for i in range(0,len(flow_temp)-2):
        flow[flow_temp[i]][flow_temp[i+1]] += cap
    return

In [227]:
def residule(graph, flow):
    cap = flow[-1]
    for i in range(0,len(flow)-2):
        graph.edge[flow[i]][flow[i+1]]['capacity'] -= cap
        if graph.edge[flow[i]][flow[i+1]]['capacity'] <= 0:
            graph.remove_edge(flow[i], flow[i+1])
        if (flow[i+1],flow[i]) not in graph.edges():
            graph.add_edge(flow[i+1],flow[i])
            graph.edge[flow[i+1]][flow[i]]['capacity'] = 0
        graph.edge[flow[i+1]][flow[i]]['capacity'] += cap
    return   

In [228]:
def process(flow , graph):
    del flow['source']
    del flow['sink']
    for key_out in flow:
        if 'sink' in flow[key_out]:
            del flow[key_out]['sink']
        if 'source' in flow[key_out]:
            del flow[key_out]['source']
        for key_in in flow[key_out]:
            if flow[key_out][key_in] > graph.edge[key_out][key_in]['capacity']:
                flow[key_out][key_in] -= flow[key_in][key_out]
                flow[key_in][key_out] = 0
    return flow      
        

In [229]:
def flow_with_demands(graph):
    """Computes a flow with demands over the given graph.
    
    Args:
        graph: A directed graph with nodes annotated with 'demand' 
        properties and edges annotated with 'capacity' 
        properties.
        
    Returns:
        A dict of dicts containing the flow on each edge. For instance, 
        flow[s1][s2] should provide the flow along edge (s1, s2).
        
    Raises:
        NetworkXUnfeasible: An error is thrown if there is no flow 
        satisfying the demands.
    """
    # TODO: Implement the function.
    # Check the condition
    i = 0
    for key in graph.node:
        i += graph.node[key]['demand']
    if (i != 0):
        raise nx.NetworkXUnfeasible()
    # Makes it a max flow problem
    graph_copy = graph.copy()
    graph_copy = preprocess(graph_copy)
    residule_graph = graph_copy.copy()
    # Initialize a zero flow
    flow = zero_flow(graph_copy)
    # find a s-t path
    flow_temp = s_t_path(graph_copy)
    # Update residule graph and 
    while flow_temp[-1] != 0:
        residule(residule_graph, flow_temp)
        add_flow(flow,flow_temp)
#         print residule_graph.edge['IN']['OH']['capacity']
#         print residule_graph.edge['OH']['IN']['capacity']
        flow_temp = s_t_path(residule_graph)
    for key in flow['source']:
        if flow['source'][key] != -graph_copy.node[key]['demand']:
            raise nx.NetworkXUnfeasible() 
    process(flow, graph)
    return flow    

To verify that your solution is correct, implement a function that computes the total flow into each node (which will be negative for supply nodes).

In [230]:
def divergence(flow):
    """Computes the total flow into each node according to the given flow dict.
    
    Args:
        flow: the flow dict recording flow between nodes.
        
    Returns:
        A dict of the net flow into each node.
    """
    # TODO: Implement the function.
    output = {}
    for key_out in flow:
        output[key_out] = 0
    for key_out in flow:
        for key_in, value in flow[key_out].iteritems():
            output[key_out] -= value
            output[key_in] += value
    return output

The following code performs a sanity check on your function (but does not completely confirm correctness).

In [231]:
flow = flow_with_demands(G)
div = divergence(flow)
print "Flow satisfies all demands:", all(div[n] == G.node[n]['demand'] for n in G.nodes())

Flow satisfies all demands: True
